In [55]:
import os
import logging 
import re


In [58]:
#设置日志记录 LLM --prompt question + response 记录
def setup_logger():
    """Setup logger to always print time and level."""
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S')
    logging.getLogger().setLevel(logging.INFO)  # logging.DEBUG

def log_w_indent(text, indent, flag, symbol='>>'):
    """Log and add indent."""
    ind2col = {i: f"\x1b[{a}" for i, a in enumerate([
        '1m', '31m', '33m', '34m', '35m'])}
    reset = "\x1b[0m"

    if indent < 0 or indent >= len(ind2col):
        raise ValueError("Invalid indent level")

    prefix = ''
    if indent > 0:
        if flag == 'Question':
            prefix = '  Q:'
        elif flag == 'Response':
            prefix = '  LLM-A:'
        elif flag == 'Code':
            prefix = '  Code script test:'
        else:
            raise ValueError("Invalid flag")

        prefix += ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset
    else:
        prefix = ind2col[indent] + text + reset

    logging.info(prefix)

setup_logger()
log_w_indent('This is a test message',2,'Question')
log_w_indent('Another message without indent', 1,'Response')

INFO:root:  Q:>>>>>>>> This is a test message
INFO:root:  LLM-A:>>>> Another message without indent


In [54]:
#配置openai客户端
from openai import OpenAI
# from openai.error import APIConnectionError
#from dotenv import load_dotenv
import openai
import os
import json
#config.json 进行配置用户openai账户 
with open(r'D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\llm_config.json') as config_file:
    config = json.load(config_file)

CLIENT = OpenAI(base_url=config['OPENAI_API_BASE'],
                api_key=config['OPENAI_API_KEY'])

###任务描述
###分步骤描述
###工具插件描述
/*通过清晰地分步骤描述任务，并提供必要的工具插件信息，有效利用LLM完成具体任务，生成每一步的代码。*/
/*确保输入结构清晰，详细描述每个步骤的要求和工具的使用方法。使得模型的输出将更加准确和实用。*/

whole-> 3个stage 实现
FOR EACH STAGE:
  1/输入初步任务描述及可使用工具信息
      - llm返回任务基本描述及规划步骤
      - 人工修改步骤并确认
  2/每一步骤逐步向llm输入并获取response代码 
      - 代码块增量进行测试
      - 代码在10个用例上测试通过进行步骤代码留存
      - 不通过，返回报错 修改代码
  3/迭代完单stage 进行批量数据测试

In [51]:
# 分离文本+提取代码块
response = """
Here is an example of Python code:

```python
import pandas as pd

# Read CSV file
data = pd.read_csv('input.csv')
print(data.head())```
You can use the above code to read a CSV file.

Another code snippet:

python
复制代码
from data_cleaning_tool import clean_data

# Clean data
cleaned_data = clean_data(data)
print(cleaned_data.head())
Make sure to install the necessary libraries before running the code.
"""


def extract_code_blocks(response):
    # 正则表达式匹配代码块
    code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
    code_blocks = code_pattern.findall(response)
    non_code_content = code_pattern.sub('', response)
    non_code_content = non_code_content.strip()
    
    return code_blocks, non_code_content

# 使用函数提取代码块和其他内容
code_blocks, non_code_content = extract_code_blocks(response)

print("Code Blocks:")
for i, code in enumerate(code_blocks, 1):
    print(f"Code Block {i}:\n{code}\n")

print("Non-Code Content:")
print(non_code_content)


Code Blocks:
Code Block 1:
import pandas as pd

# Read CSV file
data = pd.read_csv('input.csv')
print(data.head())

Non-Code Content:
Here is an example of Python code:


You can use the above code to read a CSV file.

Another code snippet:

python
复制代码
from data_cleaning_tool import clean_data

# Clean data
cleaned_data = clean_data(data)
print(cleaned_data.head())
Make sure to install the necessary libraries before running the code.


In [ ]:
import hashlib
from collections import defaultdict
import subprocess

class SpoofData:
    def __getitem__(self, item):
        return f'<{item}>'
def md5hash(string):
    return int(hashlib.md5(string.encode('utf-8')).hexdigest(), 16)
#llm问答
class LLMResponse:
    def __init__(self,ini_task:str,n_regenerate,reset,reset_stages,code_save_dir):
        self.ini_task_question=ini_task
        self.n_regenerate=n_regenerate
        self.reset=reset
        self.reset_stages=reset_stages
        self.export_predict_response = defaultdict(defaultdict(list).copy)
        self.code_scripts=None
        self.code_save_dir = code_save_dir
        
        
    #indent用于日志记录 stepid:步骤id  reuse:是否进行再次输入迭代
    def llw_w_log(self, prompt, indent,stepidx, stage,reuse=False):
        #reset.stages 重置阶段列表     拿到需要重新run的stepidx
        if (stage in self.reset_stages) and (reset := self.reset.get(stepidx, {})):
            if self.accept_restore_failure and not len(reset[md5hash(prompt)]):
                logging.warning('Spoofing aborted!')
                predict_response =llm_w_log(prompt, indent)
            else:
                # Re-Use predictions from previous runs.
                #def log_w_indent(text, indent, flag ,symbol='>>'):
                log_w_indent(f'Spoofed Input: {prompt}', indent,flag='Question')
                predict_response= reset[md5hash(prompt)].pop(0)
                log_w_indent(f'Spoofed Output: {predict_response}', indent, flag='Question',symbol='xx')
        # elif reuse and (previous := self.export_predictions.get(stepidx, {}).get(md5hash(prompt), False)):
        #     # Usually don't want to do this, but for some yes-no questions,
        #     # there is basically no variance and it saves significant compute
        #     # to just repeat the previous answer (e.g. for entailment checks).
        #     log_w_indent(f'Reused Input: {prompt}', indent)
        #     prediction = previous[0]
        #     log_w_indent(f'Reused Output: {prediction}', indent, symbol='xx')
        else:
            predict_response = llm_w_log(prompt, indent)

        self.export_predict_response[stepidx][md5hash(prompt)].append(predict_response)

        return predict_response

    def gen_ini_task_decription(self, data):
        del data
        #return 'Please list the specific factual propositions included in the answer above. Be complete and do not leave any factual claims out. Provide each claim as a separate sentence in a separate bullet point.'
        return 'For this neuroscience image task, please develop a programming task solution plan and divide your thinking into three parts: "# # Task Objectives and Function Description", "# # # Algorithm Steps and Implementation Details", and "# # Specific Constraints". There is no need to write scripts and Provide each claim as a separate sentence in a separate bullet point.'
    
    def gen_task_step(self,data):
        del data
        return''
        return 'Please list the specific step propositions included in the answer above. '
        step_description =data
        self.step_description = step_description
        return step_description
    
    def process_step_description(self)-> list:
        step_prompt_list=self.step_description.split()
       #调用template.json

        return step_prompt_list
    

    # 生成step列表 eachstep 进行prompt修饰后 进行llm输入，获取处理代码
    # manual 
    def manual_modify_step(self,data) -> list:
         ###human modify step_list###
        del data
        step_prompt_list = process_step_description(data)

        step_modify_list = []
        print("Original step_prompt_list:")
        for i, step in enumerate(step_prompt_list):
            print(f"{i+1}. {step}")
        print("\nPlease enter the modified steps. Press Enter to keep the original step.\n")

        # 循环获取用户输入
        for step in step_prompt_list:
            modified_step = input(f"Modify the step '{step}': ")
            if modified_step.strip():  # 如果输入不为空，去除前后的空格
                step_modify_list.append(modified_step)
            else:  
                step_modify_list.append(step)

        return step_modify_list

       
    def extract_code_blocks(self,response):
    # 正则表达式匹配代码块
        code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
        code_blocks = code_pattern.findall(response)
        non_code_content = code_pattern.sub('', response)
        non_code_content = non_code_content.strip()
        self.code_script = code_blocks
    
        return code_blocks, non_code_content
    
    def get_all_prompts_for_log(self):
        # Spoof data to log prompting format.
        data = SpoofData()
        prompts = dict(
            gen_ini_task_decription=self.gen_ini_task_decription(data),
            task_step_list= self.manual_modify_step(data),
            step_code =  self.extract_code_blocks(data)
            )
            # base_gen_questions=self.base_gen_questions(data),
            # base_answer_question=self.base_answer_question(data),
            # base_equivalence=self.base_equivalence(data))

        return prompts

    def run_script(self, input_data, step_id, output_suffix):
        code = self.code_script
        script_dir = self.code_save_dir
        file_path = os.path.join(script_dir, f'{step_id}_script.py')
        output_file_path = os.path.join(script_dir, f'{step_id}_output.{output_suffix}')
        
        try:
            with open(file_path, "w") as file:
                file.write(code)
        except Exception as e:
            return f"Error writing to file: {e}"

        cmd_command = f"python {file_path} {input_data}"
        try:
            result = subprocess.run(cmd_command, shell=True, capture_output=True, text=True)
            result.check_returncode()  # 检查命令是否成功运行
            logging.info(f"Code script generated successfully!!{file_path}:{result.stdout}")
            return result.stdout
        except subprocess.CalledProcessError as e:
            logging.info(f"Error running script: {e.stderr.strip()}")
            return e.stderr.strip()
        except Exception as e:
            logging.info(f"Unexpected error: {str(e)}")
            return str(e)

    
 


In [45]:
import openai

# 配置api日志记录
logging.basicConfig(level=logging.WARNING, filename='chat_llm.log', filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s')
from tenacity import (retry, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=10))


def chat_llm(prompt):
    """Predict with GPT-4 model."""

    if isinstance(prompt, str):
        messages = [
            {"role": "user", "content": prompt},
        ]
    else:
        messages = prompt

    try:
        output = CLIENT.chat.completions.create(
            model='gpt-4o',
            messages=messages,
        )
        response = output.choices[0].message.content
        return response
    except openai.error.APIConnectionError as e:
        # 记录日志警告及错误信息
        logging.warning(f"APIConnectionError occurred: {e}")
        return "An external error occurred while trying to connect to the API. Please try again later."


def llm_w_log(prompt):
    """ log inputs and outputs."""
    log_w_indent(f'Input: {prompt}', indent=1 , flag='Question')
    response = chat_llm(prompt)
    ##处理response
    log_w_indent(f'Output: {response}', indent=2, symbol='xx',flag='Response')
    return response
llm_w_log('how to trace and restruct human neuron structure based on 3d tif images?')

INFO:root:  Q:>>>> Input: how to trace and restruct human neuron structure based on 3d tif images?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:  LLM-A:xxxxxxxx Output: Tracing and reconstructing human neuron structures from 3D TIFF (Tagged Image File Format) images is a complex process that typically involves image processing, segmentation, and 3D visualization techniques. Here are the general steps you can follow, broken down into core components and tools you will need:

### Materials and Tools
- **3D TIFF images**: High-resolution images of the neuron structures.
- **Image analysis software**: For example, `ImageJ/Fiji`, `Neuroph`, `ITK-SNAP`, or `Imaris`.
- **Neuron tracing software**: `NeuronStudio`, `NeuroLucida`, `Simple Neurite Tracer` (a plugin for ImageJ), or `Horton`.
- **Programming Environment**: For advanced customization, tools like Python with libraries such as `scikit-image`, `numpy`, and `matplotlib`, or MATLAB.

'Tracing and reconstructing human neuron structures from 3D TIFF (Tagged Image File Format) images is a complex process that typically involves image processing, segmentation, and 3D visualization techniques. Here are the general steps you can follow, broken down into core components and tools you will need:\n\n### Materials and Tools\n- **3D TIFF images**: High-resolution images of the neuron structures.\n- **Image analysis software**: For example, `ImageJ/Fiji`, `Neuroph`, `ITK-SNAP`, or `Imaris`.\n- **Neuron tracing software**: `NeuronStudio`, `NeuroLucida`, `Simple Neurite Tracer` (a plugin for ImageJ), or `Horton`.\n- **Programming Environment**: For advanced customization, tools like Python with libraries such as `scikit-image`, `numpy`, and `matplotlib`, or MATLAB.\n- **High-performance Computing (HPC)** (if dealing with very large datasets).\n\n### Steps to Trace and Reconstruct Neuron Structures:\n\n#### 1. Pre-processing the Image:\nBefore segmentation and tracing, enhance th